In [78]:
import json
import time
from datetime import datetime
import requests 
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import pickle 
import os
import re
from nltk import metrics, stem, tokenize
import math


# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/


# EC2 Instance, Copy entire notebook into a .py file, send to Jerry 

In [79]:
# CONFIG_FILE = Path('.')/Path('0-setup')/Path('fec_config.yaml')
# with open(CONFIG_FILE, 'r') as f:
#     config = yaml.load(f)

In [80]:
#api_key = config['fec']['api_key']
api_key = 'IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW'

# Read In Collected DataFrames

In [81]:
df_candidate_id = pd.read_pickle('candidate_ids.pkl')
df_candidate_id.head()
df_candidate_id.name = np.array([x.lower().replace(",", "") if isinstance(x, str) else x for x in df_candidate_id.name.values])
df_candidate_id.head()



,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,aarestad david,H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,aaron laura davis,P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,abair peter jon,H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [6]:
#Reformat df_candidate_id based on cycles
len_reform_df = sum(df_candidate_id['cycles'].apply(lambda row: len(row)))
print('Length of new datframe {}:'.format(len_reform_df))



def reformat_by_cycle(df):
    df_temp = df.copy()
    
    # Create the DataFrame with 54446 rows and columns -- all NaNs
    cols = df_temp.columns
    df_cand = pd.DataFrame(np.full((len_reform_df, len(cols)), np.nan), columns=cols)
    df_cand['cycles'] = df_cand['cycles'].astype(list)
    df_cand['election_years'] = df_cand['election_years'].astype(list)
    
    i = 0
    j= 0
    for list_cycles in df_temp.cycles:
        for cycle in list_cycles:
            df_cand.iloc[j,:] = df_temp.iloc[i,:] 
            df_cand.iloc[j, 6] = cycle
            j+=1
    
        i+=1
    return df_cand

Length of new datframe 54446:


In [163]:
df_cand = reformat_by_cycle(df_candidate_id)

In [225]:
df_cand.to_pickle('df_cand.pkl')

In [82]:
df_cand = pd.read_pickle('df_cand.pkl')

In [83]:
df_cand.head()

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2012,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2014,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
2,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2016,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
3,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2018,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
4,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,2012,[2012],2012-02-22,2013-04-26T09:04:30


## Select Candidates that are involved in races we have data for. 

#### House Winners, Senate Winners, Current Governors 

In [260]:
house_winners = pd.read_pickle(Path('.')/'..'/'data'/'cleaned'/'HousePollwithWinner.pkl')
senate_winners = pd.read_pickle(Path('.')/'..'/'data'/'cleaned'/'SenatePollwithWinner.pkl')
gov_winners = pd.read_pickle(Path('.')/'..'/'data'/'cleaned'/'GovernorPollwithWinner.pkl')


In [266]:
house_winners.head()
senate_winners.head()
gov_winners.head()

df_races = house_winners.append(senate_winners)
df_races = df_races.append(gov_winners)
print(len(df_races.race_name))
df_races.head()


6638


,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,district
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20,critz,12
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22,guinta,1
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22,bass,2
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30,guinta,1
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30,bass,2


In [264]:
len(df_race_candidates[df_race_candidates.winner == '<BLANK>'])

2534

In [613]:
df_race_candidates = df_races[df_races.winner != '<BLANK>']
df_race_candidates

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,district
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20,critz,12
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22,guinta,1
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22,bass,2
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30,guinta,1
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30,bass,2
5,maryland 1st district,house,MD,"[harris, kratovil]",pos (r),"{'harris': 39.0, 'kratovil': 36.0}",harris,3.0,2010-05-01,harris,1
6,pennsylvania 12th district special election,house,PA,"[burns, critz]",r2000/daily kos (d),"{'burns': 46.0, 'critz': 40.0}",burns,6.0,2010-05-01,critz,12
7,pennsylvania 12th district special election,house,PA,"[burns, critz]",r2000/daily kos (d),"{'critz': 40.0, 'burns': 46.0}",burns,6.0,2010-05-01,critz,12
8,hawaii 1st district special election,house,HI,"[djou, case]",honolulu advertiser,"{'djou': 36.0, 'case': 28.0, 'hanabusa': 22.0}",djou,8.0,2010-05-02,hanabusa,1
9,pennsylvania 12th district special election,house,PA,"[critz, burns]",susquehanna,"{'critz': 44.0, 'burns': 38.0}",critz,6.0,2010-05-12,critz,12


In [116]:
# Load in candidate_race information
# df_race_candidates = pd.read_pickle(Path('.')/'..'/'data'/'cleaned'/'poll_race_candidates.pkl')
# df_race_candidates.head() #house, senate, governor Race Candidates 
# df_race_candidates.dropna()
# df_race_candidates.head()

,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30


### Condition on these candidates only 

In [86]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [87]:
# matche based on last names of candidates, race, and on the date 


In [275]:
cols = np.append(df_cand.columns.values, np.array(['race_candidates', 'date_of_poll', 'election_result']))
dimension_length = 2*len(df_race_candidates.race_type) + 20 # To account for potential extra rows 
df_selected_candidates = pd.DataFrame(np.full((dimension_length, len(cols)), np.nan), columns=cols)
df_selected_candidates['election_years'] = df_selected_candidates['election_years'].astype(list)

j = 0
for index, race_row in df_race_candidates.iterrows():
    
    race_type = race_row.race_type
    race_date = race_row.date
    race_candidates = race_row.top_candidates
    race_state = race_row.state
    
    for index, row in df_cand.iterrows():
        
        last_name = row[0].split()[0]
        year = row.cycles
        race = row.office_full.lower()
        state = row.state

        for race_candidate in race_candidates:
            score = fuzz.token_sort_ratio(last_name, race_candidate)
            if (score >= 100) & (race == race_type) & (year == race_date.year) & (state == race_state):
                temp_row = df_cand.iloc[index, :]
                temp_row['race_candidates'] = race_candidates
                temp_row['date_of_poll'] = race_row.date
                temp_row['election_result'] = race_row.winner
                df_selected_candidates.iloc[j,:] = temp_row
                print(j)
                j += 1
            
df_selected_candidates 
        
            
        
    



0
1


KeyboardInterrupt: 

In [279]:
df_selected_candidates = df_selected_candidates.dropna()
df_selected_candidates.head()

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date,race_candidates,date_of_poll,election_result
0,burns timothy raymond,H0PA12090,PA,Challenger,REPUBLICAN PARTY,House,2010.0,[2010],2010-03-29,2011-02-24T10:53:11,"[burns, critz]",1.271722e+18,critz
1,critz mark,H0PA12132,PA,Challenger,DEMOCRATIC PARTY,House,2010.0,"[2010, 2012]",2011-09-20,2013-04-26T09:04:30,"[burns, critz]",1.271722e+18,critz


In [ ]:
df_selected_candidates.to_pickle('df_selected_cand.pkl')

In [564]:
df_selected_candidates = pd.read_pickle('df_selected_cand.pkl')

In [565]:
df_selected_candidates.head()

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date,race_candidates,date_of_poll,election_result
0,burns timothy raymond,H0PA12090,PA,Challenger,REPUBLICAN PARTY,House,2010.0,[2010],2010-03-29,2011-02-24T10:53:11,"[burns, critz]",1.271722e+18,critz
1,critz mark,H0PA12132,PA,Challenger,DEMOCRATIC PARTY,House,2010.0,"[2010, 2012]",2011-09-20,2013-04-26T09:04:30,"[burns, critz]",1.271722e+18,critz
2,guinta frank,H0NH01217,NH,Incumbent,REPUBLICAN PARTY,House,2010.0,"[2010, 2012, 2014, 2016]",2016-10-17,2017-02-23T14:47:46,"[guinta, shea-porter]",1.271894e+18,guinta
3,shea-porter carol,H6NH01230,NH,Incumbent,DEMOCRATIC PARTY,House,2010.0,"[2006, 2008, 2010, 2012, 2014, 2016, 2018]",2016-12-03,2017-05-05T21:04:59,"[guinta, shea-porter]",1.271894e+18,guinta
4,swett katrina,H0NH02215,NH,Challenger,DEMOCRATIC PARTY,House,2010.0,"[2010, 2012]",2011-06-10,2013-04-26T09:04:30,"[bass, swett]",1.271894e+18,bass


## Load in Other DataSets

In [566]:
df_committee_id = pd.read_pickle('committee_ids.pkl')
df_committee_id.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000000
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000000
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,CA,inf
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-11-02,11/06/18 VICTORY FUND,MS,inf
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-21,1106 FUND,DC,inf


In [567]:
df_comm = pd.read_pickle('committee_id_formatted_v1.pkl')
df_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000
1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000
2,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
3,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
4,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125


# Functions to Collect Donor Information for Candidates

In [568]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
#     print(donor_cand_size.status_code)
    if donor_cand_size.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    #print(donor_cand_state.status_code)
    if donor_cand_state.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [569]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    #print(donor_comm_size.status_code)
    if donor_comm_size.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    #print(donor_comm_state.status_code)
    if donor_comm_state.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [570]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [722]:
# Input: Candidate Name, Election Type, Date-time of election (Use Pandas Date time Format)
# 1. Will incorporate a funcation call to 
# 2. to retrieve the fraction of committee donations for that candidate

# by_size/by_candidate
#  -0    $200 and under Level 1 
#  -200  $200.01 - $499.99 Level 2 
#  -500  $500 - $999.99 Level 3 
#  -1000 $1000 - $1999.99 Level 4
#  -2000 $2000 + Level 5

# Election Result: 
#     0 Lose
#     1 Win
#     3 No Data

a = cand_donor_state('P80001571', 2016)
states = a.state_full


# Returns rows, 1 row per cycle with all features for one candidate. 
def candidate_donor_df(candidate_name, states, df_candidate_id):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full',
                 'party', 'committee_id', 'committee_name','race_candidates', 'election_result']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    cols = np.append(cand_info, states)
    cols = np.append(cols, donation_levels)

    df = pd.DataFrame(np.full((1000, len(cols)), np.nan), columns=cols)
    cand_id = cand_ids[0]
#     for cand_id in cand_ids:
    #print(cand_id)

    cycles = int(cand_row.cycles.values[0])




    #Iterate through candidates cycles, add all cycles as new rows. Append to each column each

    #print(cycles)
#         for cycle in cycles:
    df_size = cand_donor_size(cand_id, cycles)
    df_state = cand_donor_state(cand_id, cycles)

    if len(df_size) > 0 or len(df_state) > 0:
        
        if df_size.values.size == 0 or df_state.values.size == 0:
                return df.dropna()
                
        if df_size.iloc[0,0] == 'time_out' or df_state.iloc[0,0] == 'time_out':
            print('API Time Out, Sleep for 1 hour')
            time.sleep(3610)

    if df_size.values.size == 0 or df_state.values.size == 0:
        return df.dropna()
    donor_states = df_state.state_full

    #Build row of information (donor_states)

    # Idea
    # Create dictionary with states as keys and initialized with 0s as values. 
    initialize_array = np.zeros(len(states))
    state_dict = dict(zip(states, initialize_array))

    for ds in donor_states:            
        # Overwrite select values with information from df_state
        state_dict[ds] = df_state[df_state.state_full == ds].total.values[0]


    # Add size features to state_dict 


    level_array = np.zeros(5)
    level_dict = dict(zip(donation_levels, level_array))

    for level in df_size['size']:
        if level == 0:
            donation_level = 'Donation Level 1'
        if level == 200:
            donation_level = 'Donation Level 2'
        if level == 500:
            donation_level = 'Donation Level 3'
        if level == 1000:
            donation_level = 'Donation Level 4'
        if level == 2000:
            donation_level = 'Donation Level 5'
        level_dict[donation_level] = df_size[df_size['size'] == level].total.values[0]


    cand_info_dict = {'name': candidate_name, 
                      'cand_id': cand_id, 
                      'state': cand_row.state.values[0],
                      'cycle': cycles,
                      'incumbent': cand_row.incumbent_challenge_full.values[0],
                      'office_full': cand_row.office_full.values[0],
                      'party': cand_row.party_full.values[0],
                      'committee_id': 0, 
                      'committee_name': 0,
                      'race_candidates': cand_row.race_candidates.values[0],
                      'election_result': cand_row.election_result.values[0]}
    state_dict = merge_two_dicts(cand_info_dict, state_dict)
    state_dict = merge_two_dicts(state_dict, level_dict)
    #return state_dict
    #df.iloc[j]
    #print(np.array(list(state_dict.values())))
    temp = list(state_dict.values())
    df.iloc[j] = np.array(temp, dtype=object) # An array of the row information
    j += 1
    return df.dropna()

                         
                         
        

In [712]:
states
candidate_donor_df('burns timothy raymond', states, df_selected_candidates)


,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,committee_name,race_candidates,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,burns timothy raymond,H0PA12090,PA,2010.0,Challenger,House,REPUBLICAN PARTY,0.0,0.0,"[burns, critz]",...,8850.0,2000.0,100.0,3150.0,0.0,529143.7,102021.27,121850.17,203654.01,401650.0


In [713]:
df_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000
1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000
2,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
3,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
4,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125


In [728]:
#Add df_candidate_id input to the previous function for candidate donor information collection function 

def get_committee_candidates(candidate_name, states, df_candidate_id, df_comm):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 
                 'committee_id', 'committee_name', 'race_candidates', 'election_result']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    cols = np.append(cand_info, states)
    cols = np.append(cols, donation_levels)

    df = pd.DataFrame(np.full((1000, len(cols)), np.nan), columns=cols)
    cand_id = cand_ids[0]
#     for cand_id in cand_ids:
    comm_rows = df_comm[df_comm.candidate_ids == cand_id]
    #return comm_rows

    #for row in comm_rows: 
    for index, row in comm_rows.iterrows(): #Iterate through each row of the dataframe comm_row
        #print('iter')
        candidate_donor_weight = row.candidate_donor_weight
        cycles = row.cycles
        comm_id = row.committee_id
        #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
        #print(cycles)
#         cycle = cycles[0]
        for cycle in cycles:
            df_size = comm_donor_size(comm_id)
            df_state = comm_donor_state(comm_id)

            
            if len(df_size) > 0 or len(df_state) > 0:
                
                if df_size.values.size == 0 or df_state.values.size == 0:
                    break
                
                if df_size.iloc[0,0] == 'time_out' or df_state.iloc[0,0] == 'time_out':
                    print('API Time Out, Sleep for 1 hour')
                    time.sleep(3610)

            if df_size.values.size == 0 or df_state.values.size == 0:
                break
            donor_states = df_state.state_full

            #Build row of information (donor_states)

            # Idea
            # Create dictionary with states as keys and initialized with 0s as values. 
            initialize_array = np.zeros(len(states))
            state_dict = dict(zip(states, initialize_array))

            for ds in donor_states:            
                # Overwrite select values with information from df_state
                state_row = df_state[df_state.state_full == ds]
                state_dict[ds] = state_row.total.values[0] * candidate_donor_weight


            # Add size features to state_dict 


            level_array = np.zeros(5)
            level_dict = dict(zip(donation_levels, level_array))

            for level in df_size['size']:
                if level == 0:
                    donation_level = 'Donation Level 1'
                if level == 200:
                    donation_level = 'Donation Level 2'
                if level == 500:
                    donation_level = 'Donation Level 3'
                if level == 1000:
                    donation_level = 'Donation Level 4'
                if level == 2000:
                    donation_level = 'Donation Level 5'
                size_row = df_size[df_size['size'] == level]
                level_dict[donation_level] = size_row.total.values[0] * candidate_donor_weight


            cand_info_dict = {'name': candidate_name, 
                              'cand_id': cand_id, 
                              'state': row.state,
                              'cycle': cycle,
                              'incumbent': 0,
                              'office_full': 0,
                              'party': 0,
                              'committee_id': row.committee_id,
                              'committee_name': row[11],
                              'race_candidates': 0,
                              'election_result': 0}
            state_dict = merge_two_dicts(cand_info_dict, state_dict)
            state_dict = merge_two_dicts(state_dict, level_dict)

            #return state_dict


            #return state_dict.values()
            df.iloc[j] = np.array(list(state_dict.values()), dtype = object) # An array of the row information
            j += 1
            break
    return df.dropna()
    


In [729]:
get_committee_candidates('critz mark', states, df_selected_candidates, df_comm)

,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,committee_name,race_candidates,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,critz mark,H0PA12132,PA,2012.0,0.0,0.0,0.0,C00525451,CRITZ VICTORY FUND 2012,0.0,...,0.0,0.0,10000.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,32500.000000
1,critz mark,H0PA12132,VA,2012.0,0.0,0.0,0.0,C00501825,JARED POLIS VICTORY FUND 2012,0.0,...,0.0,0.0,0.0,0.0,0.0,43.114186,11.627907,46.511628,58.139535,813.953488


## Run This

In [725]:
#Time to Run 
time = (len(df_selected_candidates.name) / 10 ) * 22 / 60 /60
print('Hours to run, not counting sleep time: {}'.format(time))

Hours to run, not counting sleep time: 3.388611111111111


In [726]:
int(df_selected_candidates.cycles[0])

2010

In [727]:
cand_info = ['name', 'cand_id', 'state','cycle', 'incumbent', 'office_full', 
             'party', 'committee_id', 'committee_name', 'race_candidates', 'election_result']
donation_levels = ['Donation Level 1',
       'Donation Level 2', 
       'Donation Level 3',
       'Donation Level 4',
       'Donation Level 5']
columns = np.append(cand_info, states)
columns = np.append(columns, donation_levels)
df = pd.DataFrame(np.full((100000, len(columns)), np.nan), columns=columns)

i = 1
j = 0 
amount = len(df_selected_candidates.name) 
print(datetime.now())

for name in df_selected_candidates.name:
    df_name = candidate_donor_df(name, states, df_selected_candidates)
    for index, row in df_name.iterrows():
        df.iloc[j] = row 
        j += 1
#     df = pd.concat([df, df_name], sort=False)

    df_name_comm = get_committee_candidates(name, states, df_selected_candidates, df_comm)
    if len(df_name_comm.name) != 0:
        for index, row in df_name_comm.iterrows():

            df.iloc[j] = row
            j+= 1
        
   
    #df = pd.concat([df, df_name_comm], sort=False)
    print('{}/{}'.format(i, amount))
    i+=1
    if i == 10:
        print(datetime.now())
        break
df

2018-11-14 00:04:12.347092
1/5545
  committee_id  count  cycle  size    total
0    C00525451    4.0   2012  2000  32500.0
1    C00525451    NaN   2012     0      0.0
  committee_id  count  cycle state     state_full    total
0    C00525451      3   2012    PA   Pennsylvania  22500.0
1    C00525451      1   2012    WV  West Virginia  10000.0
  committee_id  count  cycle  size     total
0    C00501825    2.0   2012   200    500.00
1    C00501825    4.0   2012  2000  35000.00
2    C00501825    2.0   2012  1000   2500.00
3    C00501825    4.0   2012   500   2000.00
4    C00501825    NaN   2012     0   1853.91
  committee_id  count  cycle state  state_full    total
0    C00501825      4   2012    NY    New York  31250.0
1    C00501825      2   2012    CA  California   5000.0
2    C00501825      6   2012    CO    Colorado   3750.0
2/5545
  committee_id  count  cycle  size    total
0    C00493577    2.0   2012  2000  18000.0
1    C00493577    NaN   2012     0      0.0
  committee_id  count  c

,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,committee_name,race_candidates,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,burns timothy raymond,H0PA12090,PA,2010.0,Challenger,House,REPUBLICAN PARTY,0,0,"[burns, critz]",...,8850.0,2000.0,100.0,3150.0,0.0,529143.700000,102021.270000,121850.170000,203654.010000,401650.000000
1,critz mark,H0PA12132,PA,2010.0,Challenger,House,DEMOCRATIC PARTY,0,0,"[burns, critz]",...,6550.0,0.0,50900.0,0.0,4800.0,109276.180000,115116.350000,221490.580000,344450.000000,441400.000000
2,critz mark,H0PA12132,PA,2012.0,0,0,0,C00525451,CRITZ VICTORY FUND 2012,0,...,0.0,0.0,10000.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,32500.000000
3,critz mark,H0PA12132,VA,2012.0,0,0,0,C00501825,JARED POLIS VICTORY FUND 2012,0,...,0.0,0.0,0.0,0.0,0.0,43.114186,11.627907,46.511628,58.139535,813.953488
4,guinta frank,H0NH01217,NH,2010.0,Incumbent,House,REPUBLICAN PARTY,0,0,"[guinta, shea-porter]",...,0.0,1000.0,0.0,0.0,0.0,145208.880000,88877.550000,120466.220000,190883.340000,302750.000000
5,guinta frank,H0NH01217,VA,2012.0,0,0,0,C00493577,GREAT EIGHT COMMITTEE,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2250.000000
6,guinta frank,H0NH01217,GA,2012.0,0,0,0,C00521559,NEW ENGLAND VICTORY FUND,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,333.333333,2000.000000,14000.000000
7,guinta frank,H0NH01217,VA,2012.0,0,0,0,C00497156,PATRIOT DAY 2011,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,150.000000,0.000000,0.000000
8,guinta frank,H0NH01217,GA,2016.0,0,0,0,C00571505,PIONEER PROJECT,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,181.818182,0.000000
9,guinta frank,H0NH01217,DC,2012.0,0,0,0,C00527549,WIRE TO WIRE COMMITTEE,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,50.000000,166.666667,200.000000,666.666667


In [730]:
df = df.dropna()
df

,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,committee_name,race_candidates,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,burns timothy raymond,H0PA12090,PA,2010.0,Challenger,House,REPUBLICAN PARTY,0,0,"[burns, critz]",...,8850.0,2000.0,100.0,3150.0,0.0,529143.700000,102021.270000,121850.170000,203654.010000,401650.000000
1,critz mark,H0PA12132,PA,2010.0,Challenger,House,DEMOCRATIC PARTY,0,0,"[burns, critz]",...,6550.0,0.0,50900.0,0.0,4800.0,109276.180000,115116.350000,221490.580000,344450.000000,441400.000000
2,critz mark,H0PA12132,PA,2012.0,0,0,0,C00525451,CRITZ VICTORY FUND 2012,0,...,0.0,0.0,10000.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,32500.000000
3,critz mark,H0PA12132,VA,2012.0,0,0,0,C00501825,JARED POLIS VICTORY FUND 2012,0,...,0.0,0.0,0.0,0.0,0.0,43.114186,11.627907,46.511628,58.139535,813.953488
4,guinta frank,H0NH01217,NH,2010.0,Incumbent,House,REPUBLICAN PARTY,0,0,"[guinta, shea-porter]",...,0.0,1000.0,0.0,0.0,0.0,145208.880000,88877.550000,120466.220000,190883.340000,302750.000000
5,guinta frank,H0NH01217,VA,2012.0,0,0,0,C00493577,GREAT EIGHT COMMITTEE,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2250.000000
6,guinta frank,H0NH01217,GA,2012.0,0,0,0,C00521559,NEW ENGLAND VICTORY FUND,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,333.333333,2000.000000,14000.000000
7,guinta frank,H0NH01217,VA,2012.0,0,0,0,C00497156,PATRIOT DAY 2011,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,150.000000,0.000000,0.000000
8,guinta frank,H0NH01217,GA,2016.0,0,0,0,C00571505,PIONEER PROJECT,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,181.818182,0.000000
9,guinta frank,H0NH01217,DC,2012.0,0,0,0,C00527549,WIRE TO WIRE COMMITTEE,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,50.000000,166.666667,200.000000,666.666667


In [ ]:
df.to_pickle('donor_df.pkl')

### GroupBy Reformat [This is not functioning as it should]

In [21]:
cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
sum_columns = np.append(states, donation_levels)
sum_columns

col = ['name','cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
df_grouped = df.groupby(cand_info)[sum_columns].sum()
df_grouped

Armed Forces Americas  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                          
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
                                      2014  0          0           0                  C00512632                       0.0   
                                      2016  0          0           0                  C00512632                       0.0   
                                      2018  0          0           0                  C00678300                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                       0.0   
                                            Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2014  0          0           0                  C00513580                       0.0   
                                      2016  0          0           0                  C00513580                       0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                       0.0   
                                            Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABAIR, PETER JON      H0MA01024 MA    2000  Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2002  Challenger House       REPUBLICAN PARTY   0                               0.0   
ABATE, CAMILLE M      H6NJ05155 NJ    2006  Challenger House       DEMOCRATIC PARTY   0                               0.0   
                                      2008  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABATE, FRANK G        H2NJ12036 NJ    1992  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABBAS, JEFFREY LYN    H0IA03071 IA    1990  Challenger House       REPUBLICAN PARTY   0                               0.0   

                                                                                                    Armed Forces Europe  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                        
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
                                      2014  0          0           0                  C00512632                     0.0   
                                      2016  0          0           0                  C00512632                     0.0   
                                      2018  0          0           0                  C00678300                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                     0.0   
                                            Challenger House       REPUBLICAN PARTY   0                             0.0   
                                      2014  0          0           0                  C00513580                     0.0   
                                      2016  0          0           0                  C00513580                     0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                     0.0   


In [ ]:
# df_grouped.to_pickle('final_donor_df.pkl')